In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [4]:
# Load data
file_path = Path('lending_data.csv')
df = pd.read_csv(file_path)

df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [5]:
# Fitting and encoding columns with LabelEncoder
le = LabelEncoder()

# Encoding homeowner column
le.fit(df["homeowner"])
df["homeowner"] = le.transform(df["homeowner"])

df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,1,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,1,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,2,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,1,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,0,53000,0.433962,5,1,23000,low_risk


In [6]:
# Create features
X = df.drop(columns="loan_status")

# Create target
y = df["loan_status"]
X

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,1,52800,0.431818,5,1,22800
1,8400.0,6.692,1,43600,0.311927,3,0,13600
2,9000.0,6.963,2,46100,0.349241,3,0,16100
3,10700.0,7.664,1,52700,0.430740,5,1,22700
4,10800.0,7.698,0,53000,0.433962,5,1,23000
...,...,...,...,...,...,...,...,...
77531,19100.0,11.261,1,86600,0.653580,12,2,56600
77532,17700.0,10.662,0,80900,0.629172,11,2,50900
77533,17600.0,10.595,2,80300,0.626401,11,2,50300
77534,16300.0,10.068,0,75300,0.601594,10,2,45300


In [7]:
X.describe()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,0.606144,49221.949804,0.377318,3.826610,0.392308,19221.949804
std,2093.223153,0.889495,0.667811,8371.635077,0.081519,1.904426,0.582086,8371.635077
min,5000.000000,5.250000,0.000000,30000.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,0.000000,44800.000000,0.330357,3.000000,0.000000,14800.000000
50%,9500.000000,7.172000,1.000000,48100.000000,0.376299,4.000000,0.000000,18100.000000
75%,10400.000000,7.528000,1.000000,51400.000000,0.416342,4.000000,1.000000,21400.000000
max,23800.000000,13.235000,2.000000,105200.000000,0.714829,16.000000,3.000000,75200.000000


In [8]:
# Check balance of target values
y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [9]:
# Split x and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y,
                                                   random_state=1,
                                                   stratify=y)
X_train.shape

(58152, 8)

In [10]:
scaler = StandardScaler()

# Fitting the scaler
X_scaler = scaler.fit(X_train)

# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

brf = BalancedRandomForestClassifier(n_estimators=1000, random_state=1)
brf.fit(X_train_scaled, y_train)

BalancedRandomForestClassifier(n_estimators=1000, random_state=1)

In [12]:
# Calculate the balanced accuracy score
y_pred_brf = brf.predict(X_test_scaled)
bas_brf=balanced_accuracy_score(y_test, y_pred_brf)
print(bas_brf)

0.9937351884428807


In [13]:
# Display the confusion matrix
cm_brf = confusion_matrix(y_test, y_pred_brf)
cm_df_brf = pd.DataFrame(
    cm_brf, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"]
)
cm_df_brf

,Predicted High Risk,Predicted Low Risk
Actual High Risk,622,3
Actual Low Risk,145,18614


In [14]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_brf))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.81      1.00      0.99      0.89      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



In [15]:
# List the features sorted in descending order by importance
importance = brf.feature_importances_
sorted(zip(brf.feature_importances_, X.columns), reverse=True)

[(0.18703526235640708, 'total_debt'),
 (0.17254127613882989, 'borrower_income'),
 (0.16565598628726366, 'debt_to_income'),
 (0.1624914600616813, 'interest_rate'),
 (0.15613786418656678, 'loan_size'),
 (0.12193867705419299, 'num_of_accounts'),
 (0.03158406206866748, 'derogatory_marks'),
 (0.0026154118463908595, 'homeowner')]

In [16]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(n_estimators=1000, random_state=1)
eec.fit(X_train_scaled, y_train)

EasyEnsembleClassifier(n_estimators=1000, random_state=1)

In [17]:
# Calculate the balanced accuracy score
y_pred_eec = eec.predict(X_test_scaled)
bas_eec=balanced_accuracy_score(y_test, y_pred_brf)
print(bas_eec)

0.9937351884428807


In [18]:
# Display confusion matrix
cm_eec = confusion_matrix(y_test, y_pred_eec)
cm_df_eec = pd.DataFrame(
    cm_eec, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"]
)
cm_df_eec

,Predicted High Risk,Predicted Low Risk
Actual High Risk,622,3
Actual Low Risk,118,18641


In [19]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_eec))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      1.00      0.99      0.91      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



In [20]:
# Evaluate the models
print(f"Based on my analysis, both models have the same Balanced Accuracy Score:")
print(f"Balanced Random Forest Classifier = {bas_brf}")
print(f"Easy Ensemble Classifier = {bas_eec}")

Based on my analysis, both models have the same Balanced Accuracy Score:
Balanced Random Forest Classifier = 0.9937351884428807
Easy Ensemble Classifier = 0.9937351884428807


#### Both models have the same recall scores (high risk: 1.00; low risk: 0.99; avg/total: 0.99

#### The Easy Ensamble Classifier had the best f1 score, with the high risk being 0.91 compared to 0.89 for the high risk f1 in relation to the Balanced Random Forest Classifier. 

#### The following three items are the top three features: 
    - Total Debt
    - Borrower Income
    - Debt to income ratio